<a href="https://colab.research.google.com/github/khoai0ngu/Ung-Dung-Phan-Tan/blob/main/ChordKT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import hashlib
import bisect
from typing import List, Tuple, Dict

# ========== Cấu hình ==========
M = 8              # số bit định danh (có thể thay đổi). 2^8 = 256 ID
N = 2 ** M         # không gian ID

# ========== Tiện ích ==========
def hash_to_id(s: str) -> int:
    """Băm một chuỗi (block hash) thành ID m-bit bằng SHA-1 rồi mod 2^m."""
    h = hashlib.sha1(s.encode('utf-8')).hexdigest()
    val = int(h, 16)
    return val % N

def in_mod_range(x: int, a: int, b: int) -> bool:
    """Kiểm tra x có thuộc (a, b] trên vòng modulo [0, N-1], gồm b và loại a.
       Xử lý cả khi vòng bị quay. Dùng cho logic successor."""
    if a < b:
        return a < x <= b
    elif a > b:
        return x > a or x <= b
    else:  # a == b -> toàn bộ vòng trừ a
        return x != a

# ========== Lõi Chord (mô phỏng một tiến trình) ==========
class Node:
    def __init__(self, node_id: int):
        self.id = node_id
        self.store: Dict[int, dict] = {}  # key -> block payload (dữ liệu dictionary)
        self.finger: List[Tuple[int,int]] = []  # danh sách (start, successor_id)

    def __repr__(self):
        return f"Nút({self.id})"

class VongChord:
    def __init__(self, node_ids: List[int]):
        self.node_ids: List[int] = sorted(set(node_ids))
        self.nodes: Dict[int, Node] = {nid: Node(nid) for nid in self.node_ids}
        self.rebuild_all_fingers()

    def sorted_nodes(self) -> List[int]:
        return sorted(self.node_ids)

    def successor(self, id_key: int) -> int:
        """Trả về ID nút là successor của id_key (nút đầu tiên >= id_key mod N)."""
        lst = self.sorted_nodes()
        idx = bisect.bisect_left(lst, id_key % N)
        if idx == len(lst):
            return lst[0]
        return lst[idx]

    def build_finger(self, node_id: int) -> List[Tuple[int,int]]:
        fingers = []
        for i in range(M):
            start = (node_id + 2**i) % N
            succ = self.successor(start)
            fingers.append((start, succ))
        return fingers

    def rebuild_all_fingers(self):
        for nid in self.node_ids:
            self.nodes[nid].finger = self.build_finger(nid)

    def find_key(self, key: int, start_node: int) -> Tuple[int, List[int]]:
        """Mô phỏng quá trình lookup trong Chord từ start_node để tìm successor(key).
           Trả về (nút giữ, đường đi)."""
        current = start_node
        path = [current]
        while True:
            succ = self.successor(key)
            if succ == current:
                return current, path
            # dùng finger table để đi gần hơn
            ft = self.nodes[current].finger
            moved = False
            for start, fid in reversed(ft):
                if fid == current:
                    continue
                if in_mod_range(fid, current, key):
                    current = fid
                    path.append(current)
                    moved = True
                    break
            if not moved:
                current = succ
                path.append(current)
            if len(path) > 4 * len(self.node_ids) + 10:
                return self.successor(key), path

    def store_block(self, block_hash: str, block_payload: dict, start_node: int = None) -> Tuple[int, List[int]]:
        """Lưu block: tính key, tìm nút chịu trách nhiệm, lưu vào đó."""
        key = hash_to_id(block_hash)
        if start_node is None:
            start_node = self.node_ids[0]
        holder, path = self.find_key(key, start_node)
        self.nodes[holder].store[key] = {'block_hash': block_hash, 'payload': block_payload}
        return holder, path

    def lookup_block(self, block_hash: str, start_node: int = None) -> Tuple[int, List[int], dict]:
        """Tìm block theo hash, trả về nút giữ, đường đi, và dữ liệu."""
        key = hash_to_id(block_hash)
        if start_node is None:
            start_node = self.node_ids[0]
        holder, path = self.find_key(key, start_node)
        data = self.nodes[holder].store.get(key)
        return holder, path, data

    def join_node(self, new_node_id: int):
        """Thêm nút mới và di chuyển các key mà nó trở thành successor."""
        if new_node_id in self.node_ids:
            print(f"Nút {new_node_id} đã tồn tại.")
            return
        self.node_ids.append(new_node_id)
        self.node_ids.sort()
        self.nodes[new_node_id] = Node(new_node_id)
        all_keys = []
        for nid in list(self.nodes.keys()):
            all_keys.extend([(nid, k) for k in list(self.nodes[nid].store.keys())])
        self.rebuild_all_fingers()
        moved = 0
        for old_owner, k in all_keys:
            correct = self.successor(k)
            if correct == new_node_id:
                val = self.nodes[old_owner].store.pop(k, None)
                if val is not None:
                    self.nodes[new_node_id].store[k] = val
                    moved += 1
        print(f"Nút {new_node_id} tham gia. Đã chuyển {moved} key sang nó.")
        self.rebuild_all_fingers()

    def leave_node(self, node_id: int):
        """Xóa một nút và chuyển key của nó sang successor."""
        if node_id not in self.node_ids:
            print("Nút không có trong vòng.")
            return
        succ = self.successor((node_id + 1) % N)
        data_items = self.nodes[node_id].store.items()
        for k, v in list(data_items):
            self.nodes[succ].store[k] = v
        del self.nodes[node_id]
        self.node_ids.remove(node_id)
        self.rebuild_all_fingers()
        print(f"Nút {node_id} rời đi. Đã chuyển {len(list(data_items))} key sang {succ}.")

# ========== Demo chính ==========
def main_demo():
    print("=== Demo Chord cho lưu trữ giống Blockchain ===")
    initial_nodes = [10, 70, 140, 200]
    ring = VongChord(initial_nodes)
    print("Các nút ban đầu:", ring.sorted_nodes())

    print("\nBảng finger (start -> successor):")
    for nid in ring.sorted_nodes():
        print(f"Nút {nid} fingers: {ring.nodes[nid].finger}")

    blocks = [
        ("blockA", {"tx_count": 5, "miner": "A"}),
        ("blockB", {"tx_count": 3, "miner": "B"}),
        ("blockC", {"tx_count": 12, "miner": "C"}),
        ("blockD", {"tx_count": 1, "miner": "D"}),
    ]

    print("\nLưu các block:")
    for bh, payload in blocks:
        holder, path = ring.store_block(bh, payload, start_node=initial_nodes[0])
        key = hash_to_id(bh)
        print(f" - {bh} (key={key}) được lưu ở Nút {holder}, đường đi={path}")

    print("\nTra cứu các block (bắt đầu từ nút 10):")
    for bh, _ in blocks:
        holder, path, data = ring.lookup_block(bh, start_node=10)
        key = hash_to_id(bh)
        print(f" - {bh} (key={key}) tìm thấy ở Nút {holder}, đường đi={path}, payload={data['payload']}")

    print("\nMô phỏng join: thêm Nút 50")
    ring.join_node(50)
    print("Các nút sau khi join:", ring.sorted_nodes())

    print("\nSau khi join, vị trí các block:")
    for bh, _ in blocks:
        holder, _, data = ring.lookup_block(bh, start_node=10)
        key = hash_to_id(bh)
        print(f" - {bh} key={key} hiện ở Nút {holder}")

    print("\nMô phỏng leave: xóa Nút 140")
    ring.leave_node(140)
    print("Các nút sau khi leave:", ring.sorted_nodes())

    print("\nVị trí cuối cùng của các block:")
    for bh, _ in blocks:
        holder, _, data = ring.lookup_block(bh, start_node=10)
        key = hash_to_id(bh)
        print(f" - {bh} key={key} hiện ở Nút {holder}")

if __name__ == "__main__":
    main_demo()


=== Demo Chord cho lưu trữ giống Blockchain ===
Các nút ban đầu: [10, 70, 140, 200]

Bảng finger (start -> successor):
Nút 10 fingers: [(11, 70), (12, 70), (14, 70), (18, 70), (26, 70), (42, 70), (74, 140), (138, 140)]
Nút 70 fingers: [(71, 140), (72, 140), (74, 140), (78, 140), (86, 140), (102, 140), (134, 140), (198, 200)]
Nút 140 fingers: [(141, 200), (142, 200), (144, 200), (148, 200), (156, 200), (172, 200), (204, 10), (12, 70)]
Nút 200 fingers: [(201, 10), (202, 10), (204, 10), (208, 10), (216, 10), (232, 10), (8, 10), (72, 140)]

Lưu các block:
 - blockA (key=235) được lưu ở Nút 10, đường đi=[10]
 - blockB (key=179) được lưu ở Nút 200, đường đi=[10, 140, 200]
 - blockC (key=124) được lưu ở Nút 140, đường đi=[10, 70, 140]
 - blockD (key=133) được lưu ở Nút 140, đường đi=[10, 70, 140]

Tra cứu các block (bắt đầu từ nút 10):
 - blockA (key=235) tìm thấy ở Nút 10, đường đi=[10], payload={'tx_count': 5, 'miner': 'A'}
 - blockB (key=179) tìm thấy ở Nút 200, đường đi=[10, 140, 200], pa